# Pytorch Implementation of EEGLearn - P. Bashivan

This notebook describes a short summary of Pytorch implementation of the models described in "Learning Representations from EEG with Deep Recurrent-Convolutional Neural Networks." Bashivan et al. at International conference on learning representations (2016).

The rest of the code is in the different python scripts of this repo.

All the codes have been inspired from the [original github](https://github.com/pbashivan/EEGLearn).

## Librairies Import

In [1]:
import numpy as np 
import scipy.io as sio
import torch
import os 
from os import path

import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader,random_split

from Utils import *
from Models import *

torch.manual_seed(1234)
np.random.seed(1234)

import warnings
warnings.simplefilter("ignore")

d:\antoine\documents\epfl_ma4\lab\dnn_sd\sd_dnn_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
participants = [12]

In [3]:
for participant in participants:
    if not path.exists(f"Data_LPSY/{participant}_images_time.mat"):
        print(f"Time Windom Images didn't exist need to be created for participant {participant}")
        create_img(PATH="Data_LPSY", Nelectrodes=128, participant=participant)

## Loading the original Images 
The images have directly been taken from original implementation, given that they remain the same nevermind the implementation (Pytorch, Tensorflow, Theano).

In [4]:
participant = 12

In [5]:
# Mean_Images = sio.loadmat("Data_LPSY/images.mat")["img"] #corresponding to the images mean for all the seven windows
#print(np.shape(Mean_Images)) 
Images = sio.loadmat(f"Data_LPSY/{participant}_images_time.mat")["img"] #corresponding to the images mean for all the seven windows
print(np.shape(Images)) 
Label = (sio.loadmat(f"Data_LPSY/{participant}_Labels.mat")["label"]).reshape(-1) #corresponding to the signal label (i.e. load levels).
print(np.shape(Label)) 

(5, 862, 3, 32, 32)
(862,)


In [6]:
np.transpose(Images,(1,0,2,3,4)).shape

(862, 5, 3, 32, 32)

In [7]:
Images_t = np.transpose(Images,(1,0,2,3,4))

## Maxpool CNN
Build the Max-pooling model performing a maxpool over the 5 parallel convnets.

In [8]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [9]:
EEG = EEGImagesDataset(label=Label, image=Images_t)

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [10]:
print('Begin Training')
res = TrainTest_Model(MaxCNN, Trainloader, Testloader, n_epoch=25, learning_rate=0.001, print_epoch=5, opti='Adam',
                     n_classes=2, n_window=5)

Begin Training
[5,  25]	loss: 0.161	Accuracy : 0.926		val-loss: 0.292	val-Accuracy : 0.913
[10,  25]	loss: 0.144	Accuracy : 0.941		val-loss: 0.156	val-Accuracy : 0.959
[15,  25]	loss: 0.030	Accuracy : 0.986		val-loss: 0.274	val-Accuracy : 0.924
[20,  25]	loss: 0.001	Accuracy : 1.000		val-loss: 0.253	val-Accuracy : 0.942
[25,  25]	loss: 0.000	Accuracy : 1.000		val-loss: 0.271	val-Accuracy : 0.942


## Temp CNN
FBuild the Conv1D model performing a convolution1D over the 5 parallel convnets.

In [11]:
print('Begin Training')
res = TrainTest_Model(TempCNN, Trainloader, Testloader, n_epoch=30, learning_rate=0.001, print_epoch=5, opti='Adam',
                     n_classes=2, n_window=5)

Begin Training
[5,  30]	loss: 0.161	Accuracy : 0.942		val-loss: 0.229	val-Accuracy : 0.924
[10,  30]	loss: 0.007	Accuracy : 0.997		val-loss: 0.384	val-Accuracy : 0.924
[15,  30]	loss: 0.011	Accuracy : 0.996		val-loss: 0.238	val-Accuracy : 0.936
[20,  30]	loss: 0.097	Accuracy : 0.971		val-loss: 0.388	val-Accuracy : 0.924
[25,  30]	loss: 0.116	Accuracy : 0.970		val-loss: 0.347	val-Accuracy : 0.953
[30,  30]	loss: 0.018	Accuracy : 0.996		val-loss: 0.228	val-Accuracy : 0.953


## LSTM CNN
Build the LSTM model applying a RNN over the 5 parallel convnets outputs

In [12]:
EEG = EEGImagesDataset(label=Label, image=Images_t)

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [13]:
print('Begin Training')
res = TrainTest_Model(LSTM, Trainloader, Testloader, n_epoch=30, learning_rate=0.0001, print_epoch=5, opti='Adam',
                     n_classes=2, n_window=5)

Begin Training
[5,  30]	loss: 0.133	Accuracy : 0.949		val-loss: 0.319	val-Accuracy : 0.872
[10,  30]	loss: 0.015	Accuracy : 0.999		val-loss: 0.290	val-Accuracy : 0.919
[15,  30]	loss: 0.011	Accuracy : 0.999		val-loss: 0.448	val-Accuracy : 0.895
[20,  30]	loss: 0.023	Accuracy : 0.991		val-loss: 0.439	val-Accuracy : 0.913
[25,  30]	loss: 0.002	Accuracy : 1.000		val-loss: 0.239	val-Accuracy : 0.936
[30,  30]	loss: 0.000	Accuracy : 1.000		val-loss: 0.287	val-Accuracy : 0.936


## Mix CNN
Build the LSTM model applying a RNN and a CNN over the 5 parallel convnets outputs

In [14]:
EEG = EEGImagesDataset(label=Label, image=Images_t)

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [15]:
print('Begin Training')
res = TrainTest_Model(Mix, Trainloader, Testloader, n_epoch=45, learning_rate=0.00001, print_epoch=5, opti='Adam',
                     n_classes=2, n_window=5)

Begin Training
[5,  45]	loss: 0.498	Accuracy : 0.828		val-loss: 0.465	val-Accuracy : 0.791
[10,  45]	loss: 0.208	Accuracy : 0.930		val-loss: 0.246	val-Accuracy : 0.890
[15,  45]	loss: 0.130	Accuracy : 0.949		val-loss: 0.207	val-Accuracy : 0.901
[20,  45]	loss: 0.086	Accuracy : 0.978		val-loss: 0.206	val-Accuracy : 0.901
[25,  45]	loss: 0.054	Accuracy : 0.991		val-loss: 0.222	val-Accuracy : 0.919
[30,  45]	loss: 0.030	Accuracy : 0.994		val-loss: 0.250	val-Accuracy : 0.919
[35,  45]	loss: 0.015	Accuracy : 1.000		val-loss: 0.282	val-Accuracy : 0.930
[40,  45]	loss: 0.008	Accuracy : 1.000		val-loss: 0.311	val-Accuracy : 0.930
[45,  45]	loss: 0.005	Accuracy : 1.000		val-loss: 0.336	val-Accuracy : 0.930
